# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Kafka + Watermarking)** </center>

---

**Proyecto Final**

**Fecha**: 13 mayo 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes del Equipo**: Tirzah Peniche Barba / Ana Cristina Luna Arellano / Juan Pedro Bihouet

**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

### Creacion de la conexión con el cluster de spark


Se crea una sesión de Spark con el paquete de Kafka incluido.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Arriba-Linux-Proyecto-Final") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8be4260-1ced-44a2-b4c3-4d1894bf95e0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Crear el stream desde Kafka

Se hace la conexión a Kafka para escuchar los dos topicos. Kafka entrega los datos en formato binario, por lo que se debe convertirlos. Se crearon los tópicos directamente en la consola de kafka para ahí subscribirse a la lectura de datos.

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "78a305ddc318:9093") \
                .option("subscribe", "twitter_topic, instagram_topic") \
                .option("startingOffsets", "latest") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transformar los datos a columnas de strings

Se usa .withColumn para castear el string y luego descomponerlo para obtener colúmnas útiles. Ya que todo se está usando en strings, se tomó la decisión de usar cast("int") en el único dato diferente para simplificar el proceso.

In [4]:
from pyspark.sql.functions import split, col

kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

social_df = kafka_df.select(
    split(col("value_str"), " \\| ").alias("fields")
).select(
    col("fields")[0].alias("timestamp"),
    col("fields")[1].alias("platform"),
    col("fields")[2].alias("user"),
    col("fields")[3].alias("text"),
    col("fields")[4].cast("int").alias("likes")
)


### Escribir el stream en formato Parquet (Data Lake)

Se guarda el resultado del stream como archivos Parquet en un directorio.

In [8]:
query = social_df.writeStream \
    .outputMode("append") \
    .trigger(processingTime='5 seconds') \
    .format("parquet") \
    .option("path", "/home/jovyan/notebooks/datalake/social_logs/") \
    .option("checkpointLocation", "/home/jovyan/notebooks/datalake/_checkpoints/") \
    .start()

query.awaitTermination(300)

25/05/11 22:28:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/11 22:28:54 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


False

### Preparación de Datos

In [ ]:
sc.stop()